In [52]:
from xml.etree.ElementTree import ElementTree,Element
import os
import re
import numpy as np
import pandas as pd
from pandas.tseries.offsets import *
import math
import folium
import folium.plugins as plugins
import time
import webbrowser as web
import branca
from ipywidgets import interact
from ctmnet.simulation import Simulator
from ctmnet.networkcreation import create_from_xml

def read_xml(in_path):
    tree = ElementTree()
    tree.parse(in_path)
    return tree

def find_nodes(tree, path):
    return tree.findall(path)

def if_match(node, kv_map):
    for key in kv_map:
        if node.get(key) != kv_map.get(key):
            return False
    return True

def get_node_by_keyvalue(nodelist, kv_map):
    result_nodes = []
    for node in nodelist:
        if if_match(node, kv_map):
            result_nodes.append(node)
    return result_nodes

def show_value_by_path(nodelist, tag, kv_map):
    rlt = []
    for parent_node in nodelist:
        children = list(parent_node)
        if children[0].text == kv_map:
            for child in children:
                print(child.tag+"  "+child.text)

def cal_distance(x1, x2):
    x1 = tuple(map(math.radians, x1))
    x2 = tuple(map(math.radians, x2))
    r = 6378.137*1000
    a = x1[1]-x2[1]
    b = x1[0]-x2[0]
    temp = math.pow(math.sin(a/2), 2)+math.pow(math.sin(b/2),
                                               2)*math.cos(x1[1])*math.cos(x2[1])
    return 2*math.asin(math.sqrt(temp))*r


def line_split(x1, x2, inter_d):
    total_length = cal_distance(x1, x2)
    split_n = int(total_length/inter_d+0.5)
    points = []
    if split_n >= 1:
        for i in range(1, split_n):
            points.append((x1[0]+(x2[0]-x1[0])*i/split_n,
                           x1[1]+(x2[1]-x1[1])*i/split_n))
        return points
    else:
        print('两点之间距离小于分割长度')
        points.append(((x1[0]+x2[0])/2, (x1[1]+x2[1])/2))
        return points

def normalize(array):
    return (array-array.min())/(array.max()-array.min())

sim = Simulator()

sec1 = sim.create('section', '诚信大道', {'lane_length': 50*9/1000,'lane_number': 3, 'cell_length': 0.05})
sec2 = sim.create('section', '丹阳大道1', {'lane_length': 50*13/1000,'lane_number': 3, 'cell_length': 0.05})
sec3 = sim.create('section', '丹阳大道2', {'lane_length': 50*8/1000,'lane_number': 3, 'cell_length': 0.05})
con = sim.connect(['丹阳大道1','丹阳大道2'],'诚信大道',priority=[10,11])
sim.create('arrival', '丹阳大道1', {'section_id':'丹阳大道1','volume':1750*4})
sim.create('arrival', '丹阳大道2', {'section_id':'丹阳大道2','volume':1750*4})
sim.create('departure', '诚信大道', {'section_id':'诚信大道','capacity':1750*3})

result = []
for i in range(800):
    sim.run_single_step()
    if i>=600:
        S=sim.sections
        t=[]
        t.extend(S['诚信大道'].last_sim_step_volume())
        t.extend(S['丹阳大道1'].last_sim_step_volume())
        t.extend(S['丹阳大道2'].last_sim_step_volume())
        result.append(t)

#print(result)



coors = [[(31.8978677, 118.7601263), (31.8979351, 118.7603678),
         (31.8980388, 118.7607598), (31.8980485, 118.7632792),
         (31.8980553, 118.7650395), (31.8982193, 118.7658549)],
         [(31.8876178, 118.7641973), (31.887687, 118.7644382), 
         (31.890086, 118.7632152), (31.8901653, 118.7634521),
         (31.8946196, 118.7614264), (31.8946992, 118.7616675),
         (31.8979351, 118.7603678)], [(31.8978677, 118.7601263), 
         (31.9003405, 118.759143), (31.9014192, 118.7585496),
         (31.9029152, 118.7574588), (31.9041872, 118.7567336), 
         (31.9050675, 118.7564219), (31.9073586, 118.7564738), 
         (31.9085629, 118.7559184), (31.9095856, 118.7556841)]]

for coor in coors:
    coor.sort(key=lambda x: x[0])

new_coors = []
for coor in coors:
    for x1, x2 in zip(coor[:-1], coor[1:]):
        new_coors.extend(line_split(x1, x2, 50))

print(new_coors)
label = [i for i in range(len(result))]

print(len(new_coors),len(result[0]))
data = []
df = pd.DataFrame(new_coors, columns=['lon', 'lat'])

for i in label:
    # df['volume']=volume[i*len(new_coors):(i+1)*len(new_coors)]
    df['volume']=normalize(np.array(result[i]))
    data.append(df.values.tolist())

# df['volume'] = np.random.random(0,200,len(new_coors))

mp = folium.Map(location=[df['lon'].mean(),
                          df['lat'].mean()], zoom_start=13, control_scale=True)


htmp = plugins.HeatMapWithTime(data,index=label)

# htmp = plugins.HeatMap(df,max_val=200, gradient={0.6:'green', 0.8: 'yellow', 1: 'red'})

htmp.add_to(mp)

mp.save('static_test.html')

[(31.898040416666667, 118.7611797), (31.898042033333333, 118.7615996), (31.898043649999998, 118.76201950000001), (31.898045266666667, 118.7624394), (31.898046883333336, 118.7628593), (31.8980502, 118.763719275), (31.8980519, 118.76415935), (31.8980536, 118.764599425), (31.898137300000002, 118.7654472), (31.88828675, 118.76413245), (31.888886499999998, 118.76382670000001), (31.88948625, 118.76352095), (31.890801628571428, 118.76316271428571), (31.891437957142855, 118.76287332857143), (31.892074285714283, 118.76258394285715), (31.892710614285715, 118.76229455714285), (31.893346942857143, 118.76200517142857), (31.89398327142857, 118.76171578571429), (31.895346380000003, 118.76140756), (31.89599356, 118.76114762), (31.896640740000002, 118.76088768), (31.89728792, 118.76062774), (31.898691966666664, 118.75979853333332), (31.899516233333333, 118.75947076666667), (31.90087985, 118.75884629999999), (31.901917866666665, 118.758186), (31.902416533333334, 118.7578224), (31.9035512, 118.7570962), 

In [36]:
print(len(new_coors),len(result))

30 15


In [1]:
from xml.etree.ElementTree import ElementTree,Element
import os
import re
import numpy as np
import pandas as pd
from pandas.tseries.offsets import *
import math
import folium
import folium.plugins as plugins
import time
import webbrowser as web
import branca
from ipywidgets import interact
from ctmnet2.simulation import Simulator
from ctmnet2.networkcreation import create_from_xml

def read_xml(in_path):
    tree = ElementTree()
    tree.parse(in_path)
    return tree

def find_nodes(tree, path):
    return tree.findall(path)

def if_match(node, kv_map):
    for key in kv_map:
        if node.get(key) != kv_map.get(key):
            return False
    return True

def get_node_by_keyvalue(nodelist, kv_map):
    result_nodes = []
    for node in nodelist:
        if if_match(node, kv_map):
            result_nodes.append(node)
    return result_nodes

def show_value_by_path(nodelist, tag, kv_map):
    rlt = []
    for parent_node in nodelist:
        children = list(parent_node)
        if children[0].text == kv_map:
            for child in children:
                print(child.tag+"  "+child.text)

def cal_distance(x1, x2):
    x1 = tuple(map(math.radians, x1))
    x2 = tuple(map(math.radians, x2))
    r = 6378.137*1000
    a = x1[1]-x2[1]
    b = x1[0]-x2[0]
    temp = math.pow(math.sin(a/2), 2)+math.pow(math.sin(b/2),
                                               2)*math.cos(x1[1])*math.cos(x2[1])
    return 2*math.asin(math.sqrt(temp))*r


def line_split(x1, x2, inter_d):
    total_length = cal_distance(x1, x2)
    split_n = int(total_length/inter_d+0.5)
    points = []
    if split_n >= 1:
        for i in range(1, split_n):
            points.append((x1[0]+(x2[0]-x1[0])*i/split_n,
                           x1[1]+(x2[1]-x1[1])*i/split_n))
        return points
    else:
        print('两点之间距离小于分割长度')
        points.append(((x1[0]+x2[0])/2, (x1[1]+x2[1])/2))
        return points

def normalize(array):
    return (array-array.min())/(array.max()-array.min())

sim = Simulator()
sec = sim.create('Section', '诚信大道', {'lanes_length': 50*91/1000,'lanes_number': 3, 'cell_length': 0.05})
sim.set_arrival('诚信大道',1750*0.6*3)
sim.set_departure('诚信大道',1750*3)

result = []
for i in range(800):
    sim.run_single_step()
    if i>=650:
        if i==700:
            sim.set_departure('诚信大道',1750*1)
        if i==750:
            sim.set_departure('诚信大道',1750*3)
        result.append(eval(str(sec)))





coors = [(31.9144045, 118.844825), (31.913614, 118.8424507), (31.9135673, 118.8423102),
         (31.9124163, 118.8386119), (31.911965, 118.8350022), (31.9117771, 118.8313818),
         (31.9115372, 118.827813), (31.9113112, 118.8245725), (31.9044129, 118.8045796),
         (31.904504, 118.8058724), (31.904923, 118.8072671), (31.9066525, 118.8108171),
         (31.906881, 118.8112643), (31.907261, 118.8120433), (31.9076736, 118.812889),
         (31.9084877, 118.8146257), (31.9090215, 118.8157643), (31.9098138, 118.8181263),
         (31.9100233, 118.8184251), (31.9109705, 118.8212361), (31.9112437, 118.8220944),
         (31.9111344, 118.8243689), (31.9145975, 118.8447214), (31.9137909, 118.8423298),
         (31.9137435, 118.8422515), (31.9137221, 118.8421872), (31.9125212, 118.8385742),
         (31.9121891, 118.8367086), (31.9120624, 118.8349727), (31.9120616, 118.8349589),
         (31.9118526, 118.8313749), (31.9116532, 118.8278092), (31.9115074, 118.8247118),
         (31.9114802, 118.8245342)]


coors.sort(key=lambda x: x[0])

new_coors = []
for x1, x2 in zip(coors[:-1], coors[1:]):
    new_coors.extend(line_split(x1, x2, 50))

label = [i for i in range(len(result))]

print(len(new_coors),len(result[0]))
data = []
df = pd.DataFrame(new_coors, columns=['lon', 'lat'])
volume = np.random.random(len(new_coors)*len(label))
for i in label:
    # df['volume']=volume[i*len(new_coors):(i+1)*len(new_coors)]
    df['volume']=normalize(np.array(result[i]))
    data.append(df.values.tolist())

# df['volume'] = np.random.random(0,200,len(new_coors))

mp = folium.Map(location=[df['lon'].mean(),
                          df['lat'].mean()], zoom_start=14, control_scale=True)


htmp = plugins.HeatMapWithTime(data,index=label)

# htmp = plugins.HeatMap(df,max_val=200, gradient={0.6:'green', 0.8: 'yellow', 1: 'red'})

htmp.add_to(mp)

mp.save('static_test.html')

两点之间距离小于分割长度
两点之间距离小于分割长度
两点之间距离小于分割长度
两点之间距离小于分割长度
两点之间距离小于分割长度
两点之间距离小于分割长度
两点之间距离小于分割长度
两点之间距离小于分割长度
两点之间距离小于分割长度
两点之间距离小于分割长度
两点之间距离小于分割长度
91 91


In [57]:
import branca
import os
from folium.plugins import MiniMap

m = folium.Map(location=[df['lon'].mean(),df['lat'].mean()], zoom_start=13, control_scale=False)
hm = plugins.HeatMapWithTime(
    data,
    radius=30,
    index=label,
    auto_play=True,
    max_opacity=0.3
)

# create a Figure, with a map inside.
f = branca.element.Figure()
h=folium.Map([31.8978677, 118.7601263], tiles='stamentoner', zoom_start=16)
hm.add_to(h)
h.add_to(f)

# put the figure into an IFrame.
iframe = branca.element.IFrame(width=500, height=300)
f.add_to(iframe)

# put the IFrame in a Popup
popup = folium.Popup(iframe, max_width=2650)

# create another map.
#m = folium.Map([31.9, 118.8], zoom_start=8)


# put the Popup on a marker, in the second map.
folium.Marker([31.8978677, 118.7601263], popup=popup).add_to(m)
hm2 = plugins.HeatMapWithTime(
    data,
    index=label,
    auto_play=True,
    max_opacity=0.3
)
hm2.add_to(m)
minimap = MiniMap(toggle_display=True,tile_layer='Stamen Toner')
minimap.add_to(m)
# get a map in a Popup.
m.save(os.path.join('popups+.html'))

m

In [17]:
coors = [[(31.8978677, 118.7601263), (31.8979351, 118.7603678),
         (31.8980388, 118.7607598), (31.8980485, 118.7632792),
         (31.8980553, 118.7650395), (31.8982193, 118.7658549)],
         [(31.8876178, 118.7641973), (31.887687, 118.7644382), 
         (31.890086, 118.7632152), (31.8901653, 118.7634521),
         (31.8946196, 118.7614264), (31.8946992, 118.7616675),
         (31.8979351, 118.7603678)], [(31.8978677, 118.7601263), 
         (31.9003405, 118.759143), (31.9014192, 118.7585496),
         (31.9029152, 118.7574588), (31.9041872, 118.7567336), 
         (31.9050675, 118.7564219), (31.9073586, 118.7564738), 
         (31.9085629, 118.7559184), (31.9095856, 118.7556841)]]

for coor in coors:
    coor.sort(key=lambda x: x[0])

new_coors = []
for coor in coors:
    for x1, x2 in zip(coor[:-1], coor[1:]):
        new_coors.extend(line_split(x1, x2, 50))
print(len(new_coors))

30
